In [2]:
import scipy.stats as stats
import numpy as np
import pandas as pd
from tkinter import filedialog
import matplotlib.pyplot as plt
import seaborn as sns
import os
from math import ceil
from copy import deepcopy
from rich.progress import track
from statannot import add_stat_annotation

In [16]:
# parameters
consecutive_window = 3
threshold_alpha = 1
threshold_D = 0.15 # um2/s
threshold_R2 = 0.5
threshold_step_size = 5 # pixels
disp_threshold = 10

In [3]:
def find_consecutive_true_ranges(bool_array):
    ranges = []
    n = len(bool_array)
    i = 0

    while i < n:
        if bool_array[i]:
            start = i
            while i < n and bool_array[i]:
                i += 1
            end = i - 1
            ranges.append((start, end))
        i += 1

    return ranges


def flag_tracks(df):
    track_flags = {}
    step_flags = []
    trackIDs = df["trackID"].unique()

    for track_id in trackIDs:
        track_data = df[df["trackID"] == track_id]
        step_flags_pertrack = np.repeat(False, track_data.shape[0])
        high_alpha = track_data["alpha"] > threshold_alpha
        ranges = find_consecutive_true_ranges(high_alpha)
        for start, end in ranges:
            if end - start + 1 >= consecutive_window:
                mean_D = np.mean(track_data["D"][start : end + 1])
                mean_R2 = np.mean(track_data["R2"][start : end + 1])
                mean_stepsize = np.mean(track_data["step_sizes"][start : end + 1])
                x_start = track_data["x"][start]
                x_end = track_data["x"][end]
                y_start = track_data["y"][start]
                y_end = track_data["y"][end]
                disp = np.sqrt((x_start - x_end) ** 2 + (y_start - y_end) ** 2)
                if (
                    mean_D > threshold_D
                    and mean_R2 > threshold_R2
                    and mean_stepsize > threshold_step_size
                    and disp > disp_threshold
                ):
                    track_flags[track_id] = True
                    step_flags_pertrack[start : end + 1] = True

        step_flags.extend(step_flags_pertrack)

    df_out = deepcopy(df)
    df_out["step_flag"] = step_flags

    return df_out, track_flags

In [8]:
test = [
    False,
    False,
    False,
    False,
    True,
    True,
    True,
    True,
    False,
    False,
    True,
    True,
    False,
    False,
]
consecutive_window = 3
ranges = find_consecutive_true_ranges(test)
for start, end in ranges:
    print(start, end)
    print(end - start + 1 >= consecutive_window)

4 7
True
10 11
False


In [ ]:
        # for i in range(len(high_alpha) - (consecutive_window - 1)):
        #     if all(high_alpha[i : i + consecutive_window]):
        #         if (
        #             np.mean(track_data["D"][i : i + consecutive_window]) > threshold_D
        #             and np.mean(track_data["R2"][i : i + consecutive_window])
        #             > threshold_R2
        #             and np.mean(track_data["step_sizes"][i : i + consecutive_window])
        #             > threshold_step_size
        #         ):
        #             track_flags[track_id] = True
        #             step_flags_pertrack[i : i + consecutive_window] = True

In [18]:
csv_file_path = filedialog.askopenfilename(
    title="Select CSV File",
    filetypes=(("CSV files", "*.csv"), ("All files", "*.*")),
)

In [19]:
df = pd.read_csv(csv_file_path)
df_out, track_flags = flag_tracks(df)

In [21]:
df_out.to_csv(csv_file_path[:-4] + "-wstepflags.csv", index=False)
print(len(track_flags))
for track_id, flagged in track_flags.items():
    if flagged:
        print(track_id)

1
6
